In [5]:
import numpy as np
import pandas as pd
import dijet
from itertools import product

In [19]:
# cell for generating pT, t, and z differential data

files = ['data/dsa_xi04_pTtz02_rs50_pn.npy',
         'data/dsa_xi04_pTtz035_rs50_pn.npy',
         'data/dsa_xi04_pTtz05_rs50_pn.npy']

zs = [0.2, 0.35, 0.5]
data = {}
for iz, z in enumerate(zs): data[z] = np.load(files[iz], allow_pickle=True).item()

targets = ['p', 'd', 'h']
corrs = ['<1>', '<cos(phi_Dp)>', '<cos(phi_Dp)cos(phi_kp)>', '<sin(phi_Dp)sin(phi_kp)>']

for z in zs:
    for tar in targets:
        if tar == 'd':
            p,n = 1,1
            omega_d = 0.07
            Pp = 1-1.5*omega_d
            Pn = Pp
    
        elif tar == 'h':
            p,n = 2,1
            pS,pD,pSp = 0.9,0.1,0.02
            Pp = -(4./3.)*(pD-pSp)
            Pn = pS - (1./3.)*(pD-pSp)
    
        elif tar == 'p': continue
    
        data[z][tar] = []
        for irep in range(len(data[z]['p'])):
            tar_rep = {}
            for corr in corrs:
                p_rep = data[z]['p'][irep][corr]
                n_rep = data[z]['n'][irep][corr]
                tar_rep[corr] = (Pp*p_rep + Pn*n_rep)/(p+n)

            data[z][tar].append(tar_rep)


In [20]:
ys = [0.05, 0.95]
Q2s = [1, 100]
zs = [0.2, 0.5]
ts = [0.1]
pTs = data[0.2]['pT values']
rss = [50]
lumi = 10

obs = ['ALL', 'cos(phi_Dp)', 'cos(phi_Dp)cos(phi_kp)', 'sin(phi_Dp)sin(phi_kp)']


In [21]:
header = ['Ymin', 'Ymax', 'Q2min', 'Q2max', 'PT', 'T', 'Z', 'RS', 'obs', 'lum', 'value', 'stat_u', 'sys_u', 'tar', 'norm_c']

replica = 1
itar = 'h'

for tar in targets:
    print(tar)
    # if tar != itar: continue
    pseudodata = []
    for ob in obs:
        errmod = 1
        if ob == 'ALL': 
            obl = '1'
            continue
        else: 
            obl = ob
            errmod = 2

        if tar == 'd':
            p,n = 1,1
            omega_d = 0.07
            Pp = 1-1.5*omega_d
            Pn = Pp
            errmod *= (Pp**2 + Pn**2)/((p+n)**2)
    
        elif tar == 'h':
            p,n = 2,1
            pS,pD,pSp = 0.9,0.1,0.02
            Pp = -(4./3.)*(pD-pSp)
            Pn = pS - (1./3.)*(pD-pSp)
            errmod *= (Pp**2 + Pn**2)/((p+n)**2)

        print(tar, np.sqrt(errmod))
    
        for rs, t, z, (ipT, pT) in product(rss, ts, zs, enumerate(pTs)):
            if pT >5: continue
            value = data[z][tar][replica][f'<{obl}>'][ipT]
            stat_error = np.sqrt(errmod/(data[z]['p'][replica]['denom'][ipT]*lumi))

            stat_error *= 1/0.7 # avg. pol. of proton for EIC (see 2007.07281)
            sys_error = 0.05*value
            pseudodata.append([ys[0], ys[1], Q2s[0], Q2s[1], pT, t, z, rs, ob, lumi, value, stat_error, sys_error, tar, 1.0])


    df = pd.DataFrame(pseudodata, columns=header)
    if lumi == 100:
        filenames = {'p': f'9000.xlsx', 'd': f'9001.xlsx', 'h': f'9002.xlsx'}
    elif lumi == 10:
        filenames = {'p': f'8000.xlsx', 'd': f'8001.xlsx', 'h': f'8002.xlsx'}
    else:
        raise ValueError(f'lumi {lumi} is not 10 or 100')
    print(filenames[tar])
    df.to_excel(filenames[tar], index=False)
        


p
p 1.4142135623730951
p 1.4142135623730951
p 1.4142135623730951
8000.xlsx
d
d 0.895
d 0.895
d 0.895
8001.xlsx
h
h 0.41475264084321367
h 0.41475264084321367
h 0.41475264084321367
8002.xlsx


In [22]:
# cell for generating pT, t differential data

file = 'data/dsa_xi04_pTt_rs50_pn.npy'

data = np.load(file, allow_pickle=True).item()

targets = ['p', 'd', 'h']
corrs = ['<1>', '<cos(phi_Dp)>', '<cos(phi_Dp)cos(phi_kp)>', '<sin(phi_Dp)sin(phi_kp)>']

for tar in targets:
    if tar == 'd':
        p,n = 1,1
        omega_d = 0.07
        Pp = 1-1.5*omega_d
        Pn = Pp

    elif tar == 'h':
        p,n = 2,1
        pS,pD,pSp = 0.9,0.1,0.02
        Pp = -(4./3.)*(pD-pSp)
        Pn = pS - (1./3.)*(pD-pSp)

    elif tar == 'p': continue

    data[tar] = []
    for irep in range(len(data['p'])):
        tar_rep = {}
        for corr in corrs:
            p_rep = data['p'][irep][corr]
            n_rep = data['n'][irep][corr]
            tar_rep[corr] = (Pp*p_rep + Pn*n_rep)/(p+n)

        data[tar].append(tar_rep)


In [23]:

header = ['Ymin', 'Ymax', 'Q2min', 'Q2max', 'Zmin', 'Zmax', 'PT', 'T', 'RS', 'obs', 'lum', 'value', 'stat_u', 'sys_u', 'tar', 'norm_c']

replica = 1
itar = 'h'

for tar in targets:
    print(tar)
    # if tar != itar: continue
    pseudodata = []
    for ob in obs:
        errmod = 1
        if ob == 'ALL': 
            obl = '1'
            continue
        else: 
            obl = ob
            errmod = 2

        if tar == 'd':
            p,n = 1,1
            omega_d = 0.07
            Pp = 1-1.5*omega_d
            Pn = Pp
            errmod *= (Pp**2 + Pn**2)/((p+n)**2)

    
        elif tar == 'h':
            p,n = 2,1
            pS,pD,pSp = 0.9,0.1,0.02
            Pp = -(4./3.)*(pD-pSp)
            Pn = pS - (1./3.)*(pD-pSp)
            errmod *= (Pp**2 + Pn**2)/((p+n)**2)

        print(tar, np.sqrt(errmod))
        
        for rs, t, (ipT, pT) in product(rss, ts, enumerate(pTs)):
            if pT > 5: continue
            if pT < 2: continue
            value = data[tar][replica][f'<{obl}>'][ipT]
            stat_error = np.sqrt(errmod/(data['p'][replica]['denom'][ipT]*lumi))
            stat_error *= 1/0.7 # avg. pol. of proton for EIC (see 2007.07281)
            stat_error *= 1/np.sqrt(0.2) # assume average efficiency is 20% for this measurement
            sys_error = 0.05*value
            pseudodata.append([ys[0], ys[1], Q2s[0], Q2s[1], zs[0], zs[1], pT, t, rs, ob, lumi, value, stat_error, sys_error, tar, 1.0])


    df = pd.DataFrame(pseudodata, columns=header)
    if lumi == 100:
        filenames = {'p': f'1000.xlsx', 'd': f'1001.xlsx', 'h': f'1002.xlsx'}
    elif lumi == 10:
        filenames = {'p': f'2000.xlsx', 'd': f'2001.xlsx', 'h': f'2002.xlsx'}
    else:
        raise ValueError(f'lumi {lumi} is not 10 or 100')
    print(filenames[tar])
    df.to_excel(filenames[tar], index=False)
        


p
p 1.4142135623730951
p 1.4142135623730951
p 1.4142135623730951
2000.xlsx
d
d 0.895
d 0.895
d 0.895
2001.xlsx
h
h 0.41475264084321367
h 0.41475264084321367
h 0.41475264084321367
2002.xlsx
